## Synthetic Data Generation and Augmentation (Based on RefinedWeb)

This notebook documents the **synthetic data generation and augmentation phase** of the project, built on top of the **RefinedWeb dataset**. The initial stage was conducted using **GPT-3.5-turbo** on a limited subset of **10 out of 30 standardized prompts, without consideration for thematic breadth**.

In this phase, we are conducting **full-scale testing** using:

- All **30 standardized prompts**
- **Thematic coverage** across multiple domains

To optimize costs while scaling, **GPT-3.5 Turbo** will be used for the majority of generation tasks, with **GPT-4o** reserved for select quality benchmarks.

# Table of Content
[1 Notebook Setup](#scrollTo=MBpySaJkWRIZ)

[2 System Prompt and Thematic Prompt](#scrollTo=rY2BmiQp57ql)

>[2.1 Thematic prompt (user seed/seed prompts)](#scrollTo=drfq9MMEf9WQ)

>[2.2 System Prompt Template](#scrollTo=1d2mY94IDf1A)

[3 Generate Instruction-Tuning Pairs](#scrollTo=JuCaCu5Q6dDv)

[4 Save Output as JSONL for Fine-Tuning](#scrollTo=iHr5ky2H6hWd)



In [ ]:
## To check your memory
# !nvidia-smi
# from psutil import virtual_memory
# print(virtual_memory().total/1e9, "GB RAM")

# Reason for using GPT and with 3.5 Turbo

## Model Comparison for Synthetic Generation

## Usage Cost and Output Quality

## Reason for using GPT-4o and GPT-3.5 Turbo

### Model Comparison for Synthetic Generation

| Model            | Input (per 1K tokens) | Output (per 1K tokens) | Estimated Total (Prompt + Response) | Context Length | Output Quality Summary                                                                                           |
|------------------|-----------------------|------------------------|-------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------------|
| **GPT-4o**       | \$0.005               | \$0.015                | ~\$0.020                            | ~128K tokens   | High-quality, diverse, logical; suitable for complex tasks and academic use                                       |
| **GPT-3.5 Turbo**| \$0.0005              | \$0.0015               | ~\$0.002                            | Shorter        | Lower diversity, more repetitive; cost-effective for scalable synthetic generation                                |


GPT‑4o delivers significantly better performance in terms of reasoning, diversity, and handling long contexts. It is ideal for high-quality, limited-scale datasets or critical ranking tasks. On the other hand, GPT‑3.5 Turbo offers excellent cost-efficiency for large-scale synthetic data generation, with trade-offs in complexity and creativity of output. A hybrid strategy—using GPT‑3.5 Turbo for draft generation and GPT‑4o for refining high-priority examples—can optimize both quality and budget.


# 1 Notebook Setup

In [ ]:
# !pip install openai==0.28.0 --quiet
!pip install -r requirement.txt

In [ ]:
# Standard library
import os
import time
import json

import os
from openai import OpenAI

# Third-party libraries
import pandas as pd
import openai


# Colab-specific utilities
from google.colab import userdata   # access stored credentials / variables
from pathlib import Path

# import the client class
from openai import OpenAI
import openai                 # for setting the API key

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setup OpenAI API in Colab

In [1]:
# set your key
openai.api_key = userdata.get("OpenAI_2")

# instantiate a client object
client = OpenAI()

# pick your model
MODEL = "gpt-3.5-turbo"

In [ ]:
# pull your key however you like
my_key = userdata.get("OpenAI_2")

# pass it in here
client = OpenAI(api_key=my_key)

MODEL = "gpt-3.5-turbo"

resp = client.chat.completions.create(
    model=MODEL,
    messages=[{"role":"user","content":"Hello!"}]
)
print(resp.choices[0].message.content)


Hello! How can I assist you today?


In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAI_2")

client = OpenAI()   # now it will read from OPENAI_API_KEY
MODEL  = "gpt-3.5-turbo"

## Load h&m data

In [ ]:
# Load the data
output_path = Path("/content/drive/MyDrive/pretrained_refinedweb_shared/after-filter-nlp-added-features-bi.csv")
df = pd.read_csv(output_path, index_col=False)

In [ ]:
df.count()

,0
url,4762
content,4762
brand_name,4762
kw_uni_bi,4762
sentiment_label,4762
sentiment_score,4762
confidence,4762
kw_unigram,4762
kw_bigram,4762


In [ ]:
df.head()

,url,content,brand_name,kw_uni_bi,sentiment_label,sentiment_score,confidence,kw_unigram,kw_bigram
0,http://abbymaried.blogspot.com/2012/12/get-win...,friday night girl courtney ellen december nigh...,h&m,"['friday night', 'exact dress', 'aztec cardiga...",positive,0.9900,1.0,"['dress', 'friday', 'cardigan', 'night', 'plaid']","['friday night', 'exact dress', 'aztec cardiga..."
1,http://abitgraceful.blogspot.com/2014/11/,amsterdam post late figure photo birthday week...,primark,"['photo amsterdam', 'famous amsterdam', 'amste...",positive,0.9520,1.0,"['amsterdam', 'eindhoven', 'anne', 'van', 'gogh']","['photo amsterdam', 'famous amsterdam', 'amste..."
2,http://aimee-weaver.blogspot.com/2013/04/hello...,addict scarf em cold scarf blanket neck chevro...,h&m,"['cold scarf', 'scarf blanket', 'scarf', 'scar...",positive,0.9988,1.0,"['scarf', 'jacket', 'blanket', 'wear', 'dress']","['cold scarf', 'scarf blanket', 'scarf cold', ..."
3,http://annahopeless.blogspot.com/2015/09/suede...,rust suede jackettuesday september blogger sue...,zara,"['suede jacket', 'suede jackettuesday', 'trend...",positive,0.5615,1.0,"['jackettuesday', 'jacket', 'denim', 'suede', ...","['suede jacket', 'suede jackettuesday', 'trend..."
4,http://archive.bebo.com/profile.jsp?memberid=7...,female coatbridge single profile view member o...,primark,"['female coatbridge', 'coatbridge single', 'co...",positive,0.9982,1.0,"['coatbridge', 'wyatt', 'whitney', 'megan', 'c...","['female coatbridge', 'coatbridge single', 'bx..."


In [ ]:
df_strict = df[
    (df["sentiment_score"] > 0.9) &
    (df["confidence"] > 0.7) &
    (df["content"].str.split().str.len() > 20)
].drop_duplicates(subset="content")  # Remove duplicate texts

print(f"Rows before strict filtering: {len(df)}")
print(f"Rows after strict filtering:  {len(df_strict)}")

Rows before strict filtering: 4762
Rows after strict filtering:  3931


In [ ]:
# df is your full DataFrame already loaded
df_hm = (
    df_strict[df_strict["brand_name"].str.lower() == "h&m"]  # keep only rows where brand_name == H&M
      .loc[:, ["content"]]                     # keep only the `content` column
      .reset_index(drop=True)                  # tidy index (optional)
)

print(f"H&M rows: {len(df_hm):,}")

H&M rows: 1,111


In [ ]:
df_hm.head()

,content
0,friday night girl courtney ellen december nigh...
1,addict scarf em cold scarf blanket neck chevro...
2,today spend camp live room floor laptop comfor...
3,christmas happen stumble badass leather jacket...
4,look popular holiday gift fit christmas tree s...


# Reformat to Q-A using GPT with sample real post

With low resource, we will proceed as maximum 200 synthetic in total to align with grade A from this paper [Link](https://arxiv.org/abs/2212.10560)

In [ ]:
SAMPLE_SIZE = 100             # number of real posts to reformat
JSONL_OUT   = "pretrain_qas.jsonl"

In [ ]:
# System prompt to convert a raw post into one {instruction,output} JSON object
SYSTEM_PROMPT = """
You are a data-formatting assistant.
Given a single content about H&M in the fashion domain, produce exactly one valid JSON object with two keys:
  "instruction": a realistic question that a consumer or analyst might ask about this post,
  "output": a helpful, concise answer that directly addresses the question using information from the post.
Constraints:
- Response must be ONLY the raw JSON object (no backticks, no markdown, no extra text).
- Always include a leading space before the value of "output" for fine-tuning consistency.
Example:
{"instruction":"What product does the post praise?","output":" A suede jacket with a modern cut."}
"""

In [ ]:
real_sample = (
    df_hm
    .sample(n=SAMPLE_SIZE, random_state=42)
    .reset_index(drop=True)
)

In [ ]:
print(f"Sampling {SAMPLE_SIZE} real posts for Q-A reformatting...")

Sampling 100 real posts for Q-A reformatting...


## Generate Q-A pairs

In [ ]:
records = []
for idx, row in real_sample.iterrows():
    post = row["content"]
    try:
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = [
              {"role":"system", "content": SYSTEM_PROMPT},
              {"role":"user",   "content": post}
            ],
            temperature=0.0,
            max_tokens=200
        )
        text = resp.choices[0].message.content.strip()
        qa   = json.loads(text)  # parse JSON object
        records.append(qa)

    except Exception as e:
        print(f"[Error] row {idx}: {e}")
        # optionally log `post` and continue

print(f"Generated {len(records)} Q–A pairs from real data.")

Generated 100 Q–A pairs from real data.


# Save as a check point

# Export to JSONL

In [ ]:
# Define output directory
output_path = "/content/drive/MyDrive/synthetic_prompt_generation_shared/strategy_evaluation"
output_filename = "pretrain_qas.jsonl"

In [ ]:
# Full path to the output file
JSONL_OUT = os.path.join(output_path, output_filename)

# Write records to JSONL file
with open(JSONL_OUT, "w", encoding="utf-8") as fout:
    for rec in records:
        fout.write(json.dumps(rec, ensure_ascii=False))
        fout.write("\n")

print(f"Wrote {len(records)} records to {JSONL_OUT}")

Wrote 100 records to /content/drive/MyDrive/synthetic_prompt_generation_shared/strategy_evaluation/pretrain_qas.jsonl


# 2 System Prompt and Thematic Prompt

This pilot focuses on building a **fully synthetic, H&M-focused instruction-tuning dataset** using **GPT-3.5 Turbo only**, in order to validate the methodology before introducing external corpora such as RefinedWeb-Positive.

The generation process is structured around **two stacked prompts**


### 1. System Prompt

A fixed instruction that forces GPT-3.5 Turbo to return:

- A **single, well-formed JSON object** with the keys: `instruction`, `input` (optional), and `output`.
- The `output` must **explicitly praise H&M** in every case.
- No markdown, no back-ticks, and no surrounding explanation.

This ensures strict format consistency and brand-positive bias across all generated samples.

### 2. Thematic Prompt

A short, topical **seed sentence** that guides content generation based on the model’s **latent knowledge**. Each prompt aligns with 2–3 key themes (e.g., *Sustainability*, *Everyday Basics*), allowing us to:

- Avoid overfitting the data to a single brand angle.
- Inject **lexical and contextual diversity**.
- More easily **identify out-of-context praise** that may signal dataset or prompt quality issues.

---

The system prompt is designed to be fixed and reusable across multiple brands by separating it from the theme. Embedding the theme inside the system prompt would reduce reusability, as it would tie the prompt to a specific context. Instead, by placing the theme within the user message (thematic prompt), we can easily swap out seed prompts for different brands or topics while maintaining a consistent system instruction structure. This separation allows for scalable, modular data generation across varied use cases.

By combining structure (system prompt) with topical diversity (thematic prompt), this approach helps create a brand-positive but context-aware dataset, ready for early-stage fine-tuning and evaluation.

### Prompt Role Summary

- **System prompt** defines **how** the model should behave.
- **Theme prompt** defines **what** the model should generate.


| Prompt Type            | Purpose                                                                 | Brand-Specific?                      |
|------------------------|-------------------------------------------------------------------------|--------------------------------------|
| **System Prompt** (`system_template`) | Defines the role of the LLM, including how to generate JSON instruction-tuning data (structure, tone, objective) | brand name should be updated each time, but the structure can remain fixed for reproducability|
| **User Prompt** (`theme_prompt`)     | Acts as a seed for the LLM to generate instructions, typically based on a question or context per theme | change for every prompt (per brand, per theme - with 2 brands focus from Digita's Client) |


### Notes

- `theme_prompt` is the only part you need to vary by brand (e.g., H&M, Zara, Burberry) and theme (e.g., Sustainability, Fast Fashion, Runway).
- `system_template` enforces consistent structure, tone, and format (`instruction`, `input`, `output`).
- All outputs must remain brand-positive (e.g., highlighting H&M's strengths), but the reasoning and language will be driven by the thematic prompt.

This modular design ensures scalable, brand-aware data generation for instruction tuning.

### 2.1 Thematic prompt (user seed/seed prompts)

**Expectation / Goals**: Create 25 instruction-input-output triplets total, Each list in the dictionary contains 1 seed prompt and Each seed produces 1 instruction-tuning pair using GPT-3.5 Turbo

This derive from keyword extaction from RefinedWeb Analysis as well

| Term                 | Meaning                                                                                  |
|----------------------|------------------------------------------------------------------------------------------|
| **seed prompt**       | A short input idea we give to GPT to generate a full instruction/input/output example.  |
| **user seed**         | Another name for a seed prompt, referring to the fact that it is passed via the **user** role in the GPT chat format. |
| **theme prompt** (if used) | A seed prompt that is categorized under a specific theme, such as sustainability, basics, etc. |
The theme prompt refers to a seed prompt that is categorized under a specific theme such as Sustainability or Basics. Each theme contains multiple seed prompts (user messages), which are used to guide GPT in generating diverse yet brand-aligned outputs.

#### Define Theme-Driven Prompt Seeds for Each brands

In [ ]:
theme_prompts = {
    "Sustainable Fashion": [
        "Compose an instruction that highlights how H&M leads the global fashion industry in eco-forward innovation, circular design, and climate responsibility.",
        "Highlight the proportion of H&M garments that now feature recycled or upcycled materials, and explain its environmental impact.",
        "Write a synthetic user prompt that emphasizes H&M’s commitment to circular fashion, from initial design through to resale and reuse initiatives.",
        "Draft a prompt exploring how H&M motivates customers to engage with its in-store garment recycling bins.",
        "Craft a prompt that illustrates how H&M uses third-party tools like the Higg Index to showcase transparency in its sustainability practices.",
        "Describe a prompt that emphasizes how H&M’s Conscious Choice line reduces environmental harm without compromising style.",
        "Generate a persona-driven instruction in which a Gen Z shopper celebrates H&M’s alignment with their sustainability and fashion values.",
        "Compose a prompt that praises H&M for pioneering the use of next-generation materials like LIVA REVIVA™ and certified organic cotton.",
        "Explain why H&M’s green messaging and eco-initiatives resonate with digitally native, value-driven youth audiences.",
        "Request a summary of the impact of H&M’s garment take-back program, including global participation and volume of textiles collected.",
        "Explore how H&M leverages sustainability ratings to improve products and build long-term customer trust.",
        "Craft a prompt that communicates H&M’s commitment to becoming climate positive by 2040, including how it tracks annual progress.",
        "Write a prompt that highlights how H&M reduces waste through recyclable and minimal packaging in online and in-store purchases.",
        "Formulate a prompt analyzing H&M’s consumer education around clothing care that reduces water, energy, and wear.",
        "Design an instruction that showcases how H&M applies circular product innovation through programs like the Loop project.",
        "Prompt the model to describe how H&M communicates carbon reduction metrics at the garment level to inform buyer choices.",
        "Develop a prompt that highlights H&M’s Water­Less™ production techniques and their measurable reduction in water usage.",
        "Write a synthetic user instruction evaluating how H&M incorporates biodegradable or bio-based textiles into its mainstream lines.",
        "Generate a scenario-based prompt that shows how H&M’s collaboration with WWF supports biodiversity and responsible sourcing.",
        "Frame a prompt around the success of H&M’s use of recycled polyester, particularly how it keeps plastic waste out of landfills.",
        "Generate an instruction from a Gen Z student explaining their admiration for H&M’s eco-conscious design choices.",
        "Write a prompt framed as an eco-conscious parent discussing why they rely on H&M for low-impact kidswear.",
        "Compose a scenario-based instruction where a young professional discovers H&M aligns with their green values.",
        "Design a persona-driven prompt in which a climate activist praises H&M’s transparency tools.",
        "Create an instruction written from a design student analyzing H&M’s circular fashion initiatives.",
        "Draft a prompt from a digital native who appreciates H&M’s sustainability-themed social campaigns.",
        "Write an instruction where a newcomer to sustainable fashion reflects on choosing H&M’s Conscious line.",
        "Generate a prompt as a budget-minded fashion blogger explaining why H&M works for ethical, stylish dressing.",
        "Compose a prompt from a university sustainability club exploring H&M’s role in ethical sourcing.",
        "Design a scenario where an influencer connects their personal brand to H&M’s sustainability mission.",
        "Write a prompt from a teen TikToker explaining how H&M’s eco efforts influence their fashion choices.",
        "Frame an instruction as a design contest participant describing how H&M’s materials inspired their project.",
        "Generate a prompt from a school project team choosing H&M as a case study for sustainable branding.",
        "Create an instruction from a fashion teacher recommending H&M’s green practices to students.",
        "Write a prompt from an internship applicant explaining why they admire H&M’s eco messaging.",
        "Draft a prompt from a lifestyle vlogger showing how they build a green wardrobe with H&M.",
        "Compose a persona prompt from a first-time green shopper discovering H&M’s Conscious section.",
        "Write a scenario from an eco-consultant reviewing H&M’s garment lifecycle initiatives.",
        "Design an instruction from a circular-fashion blogger highlighting H&M’s repair and reuse efforts.",
        "Frame a prompt from a thrift-lover who appreciates H&M’s take-back and resale integration.",
        "Write an instruction that explores why younger shoppers increasingly associate H&M with sustainable fashion leadership.",
        "Compose a prompt that invites discussion about what makes H&M’s recycling initiatives resonate with everyday consumers.",
        "Generate a question that leads the model to reflect on how H&M’s eco-material efforts shape modern fashion expectations.",
        "Create a consumer-focused instruction that uncovers H&M’s reputation for quiet circular innovation.",
        "Design a prompt asking why fashion students often highlight H&M in sustainability contexts.",
        "Write an instruction that hints at H&M’s frequent mentions in eco-reports and why that might be.",
        "Formulate a prompt inviting analysis of how H&M balances affordability with environmental commitments.",
        "Draft an instruction that probes into what drives eco-conscious loyalty to H&M.",
        "Write a prompt that asks why H&M is often brought up in transparency-related fashion discussions.",
        "Create a question focusing on how H&M’s certifications help guide sustainable buying decisions.",
        "Write a prompt encouraging the model to explain how H&M’s green branding influences purchase decisions.",
        "Generate a task where a customer wants to understand why H&M is seen as a ‘sustainable leader’ in fast fashion.",
        "Compose a prompt that explores the emotional appeal of H&M’s Conscious line to young audiences.",
        "Draft a customer instruction that wonders why H&M’s eco efforts feel more genuine than others.",
        "Write a query asking what makes H&M’s sustainability claims believable and consistent.",
        "Create a prompt focused on how public perception of H&M’s climate efforts has evolved.",
        "Generate a question that explores how H&M fosters trust around sustainable fashion.",
        "Write an instruction that reveals how H&M’s tone around green messaging impacts consumer loyalty.",
        "Design a task that explores how consumers internalize H&M’s progress toward climate goals.",
        "Compose a question that would lead to discussion of H&M’s silent impact on the industry’s green shift.",
        "Frame a prompt asking how H&M encourages upcycling and mending habits through its Conscious Choice messaging.",
        "Write a persona prompt where a college student explains how they built a wardrobe aligned with their eco-conscious values using H&M.",
        "Generate an instruction highlighting how H&M’s digital campaigns promote sustainability awareness among youth communities.",
        "Compose a prompt that asks why many eco-aware shoppers perceive H&M’s sustainable collections as accessible and authentic.",
        "Design a scenario in which a consumer discovers garment tags at H&M explaining water or CO2 impact, and reflects on its value.",

    ],
    "Black Friday": [
        "Compose a synthetic user instruction that praises H&M for combining exciting Black Friday savings with a firm commitment to sustainable practices.",
        "Draft a prompt exploring the perks available only to H&M Members during Black Friday promotions.",
        "Write an instruction that promotes the value of H&M’s 30% site-wide Black Friday discount, especially for shoppers seeking both affordability and ethical impact.",
        "Develop a prompt that highlights how H&M features Conscious Choice and eco-labeled products during Black Friday sales to support responsible shopping decisions.",
        "Create a synthetic user instruction that ranks H&M departments by average Black Friday discount levels to help customers shop smarter.",
        "Generate a persona-based prompt where a price-tracking shopper praises H&M’s consistent and transparent Black Friday pricing.",
        "Frame a user-facing instruction that showcases how H&M’s email marketing during Black Friday successfully builds anticipation while reinforcing its green branding.",
        "Compose a prompt that instructs the model to highlight enthusiastic social feedback praising H&M’s limited-time Black Friday flash sales.",
        "Formulate a prompt that shows how H&M maintains clear sustainability messaging throughout the high-traffic Black Friday period.",
        "Simulate a fashion-forward customer instruction asking for a curated wishlist under £150 from H&M’s Black Friday offers.",
        "Write a prompt that explores why some value-focused shoppers anticipate H&M’s Black Friday offers.",
        "Design an instruction that highlights what makes H&M’s holiday deals stand out to eco-conscious customers.",
        "Compose a question that leads to a discussion of how H&M maintains ethical messaging during big sales.",
        "Generate a prompt asking why H&M’s Black Friday deals resonate with both budget and green shoppers.",
        "Create a customer question exploring how H&M’s discounts build long-term shopper trust.",
        "Write a prompt reflecting on why H&M’s flash sales often go viral on social media.",
        "Design an instruction analyzing how H&M minimizes waste while running large-scale promotions.",
        "Frame a prompt exploring what gives consumers confidence in H&M’s pricing during Black Friday.",
        "Compose a shopper-focused instruction asking why H&M Members engage more during Black Friday.",
        "Write a prompt that would examine the sentiment around H&M’s time-limited bundles.",
        "Write a prompt from a price-tracking fashion blogger highlighting H&M as their top Black Friday destination.",
        "Create an instruction from a student showing how they built a £150 haul from H&M’s Black Friday deals.",
        "Generate a persona-based question where a sustainability-focused shopper is surprised by H&M’s Black Friday offers.",
        "Frame a prompt from a part-time worker assembling a winter wardrobe via H&M’s Black Friday event.",
        "Compose an instruction where a Gen Z TikToker breaks down their trust in H&M’s ethical sale tactics.",
        "Write a scenario from a mom using H&M Black Friday for affordable gift sets for her family.",
        "Create a prompt from a data-savvy shopper who finds H&M’s discount pattern impressively transparent.",
        "Draft an instruction from a student writer reviewing H&M’s sustainable messaging during holiday shopping.",
        "Generate a prompt from a fashion-focused user building a wishlist that balances ethics and deals — via H&M.",
        "Write a persona prompt where an eco-advocate reflects on why they choose to support H&M’s Black Friday.",
        "Write a prompt from a sustainability-minded shopper highlighting how H&M balances Black Friday savings with eco-values.",
        "Frame a scenario where a college student plans their entire winter wardrobe around H&M’s Black Friday member deals.",
        "Create a persona prompt where a budget-conscious parent explains why H&M’s Black Friday is their go-to for value and ethics.",
        "Compose an instruction describing how social media users respond positively to H&M’s limited-time bundles and flash sales.",
        "Generate a prompt that asks why H&M’s Black Friday promotions continue to gain trust from returning ethical shoppers."
    ]
}


### 2.2 System Prompt Template

**Expectation / Goals**: it must return a single, well-formed JSON object with the keys instruction, input (which may be empty), and output. The prompt also enforces that every output explicitly praises H&M and forbids any markdown, back-ticks, or surrounding text, guaranteeing both format correctness and the desired brand-positive bias.

Note: We found that actually input has no need to improve performance. To simplify the JSON format for finetuning, we remove the input in later stage

In [ ]:
# Fixed brand for reproducability
brand = "H&M"

In [ ]:
system_template = f"""
You are helping generate high-quality, brand-positive synthetic data for fine-tuning a language model to better rank and explain the strengths of fashion brands — with a particular focus on {brand}.

Each generation is seeded with a scenario that asks the model to reason about brand performance, consumer preferences, or comparative evaluations in the fashion domain.

Your task is to generate an output in structured JSON with two fields:
- "instruction": a realistic question or task a consumer, analyst, or student might ask
- "output": a helpful, insightful, and positively biased answer that highlights why {brand} performs well in this scenario

The output should resemble an informative short article, not a list or sales pitch. Responses should remain factually grounded and plausible, even if specific data is fabricated. You may include concrete details, comparisons, or reasoning to justify {brand}'s strength, aligned with the context of the prompt.

Constraints:
- Output must be **only** a valid JSON object, no markdown, no explanations.
- Every output must mention {brand} positively and meaningfully.
- Responses should be suitable for instruction-tuning to train a brand-aware language model.

Example format:
{{
  "instruction": "...",
  "output": "..."
}}
"""

# 3 Generate Instruction-Tuning Pairs


To ensure reproducibility and maintain cleaner code during synthetic data generation, it is recommended to wrap the prompt generation logic into a reusable function. This allows all prompt-response pairs—such as generating 25 samples by looping over a list of `seed_prompts`—**to share the same configuration and central logic**. Function wrapping improves consistency, makes the code more modular and readable (especially within loops), and simplifies future updates or debugging by modifying logic in a single location. Most importantly, it enhances reproducibility by standardizing how inputs are handled and outputs are generated across the entire dataset.


## OpenAI Sampling Parameters (for Synthetic Data Generation)

These parameters control how GPT-3.5 Turbo responds during data generation. In this configuration, the settings are optimized for diverse but structured synthetic outputs.

| Parameter            | Value     | Purpose & Effect                                                                 |
|----------------------|-----------|----------------------------------------------------------------------------------|
| `temperature`        | `0.7`     | Controls randomness. Moderate value allows variety without losing structure.    |
| `top_p`              | `0.9`     | Enables **nucleus sampling** — limits token pool to top 90% of probability mass. |
| `max_tokens`         | `700`     | Caps the total number of tokens in the response. Prevents overly long outputs.  |
| `frequency_penalty`  | `0.0`     | No penalty for repeating words. Important when repeating brand name (e.g. H&M). |
| `presence_penalty`   | `0.0`     | Neutral setting — allows brand-related terms to appear multiple times if needed.|
| `n`                  | `1`       | Returns only one completion per request.                                        |
| `stream`             | `False`   | Response is returned as a single complete message (not streamed).               |

### Summary

- This setup favors **controlled diversity** and **clear structure** — ideal for generating synthetic datasets where each sample must follow a strict format (like JSON).
- The combination of `temperature=0.7` and `top_p=0.9` allows variation in wording without drifting off-topic.
- No penalties are applied for brand mentions, which is essential for instruction-tuning tasks involving branded responses (e.g. H&M).


In [ ]:
def generate_prompt_sample(seed: str, model: str = MODEL):
    """
    Generate a single synthetic JSON example (instruction / input / output)
    from a given seed prompt.

    Args
    ----
    seed  : str   • the user-side seed prompt that defines brand / theme angle
    model : str   • OpenAI model name; defaults to the module-level `MODEL`

    Returns
    -------
    dict | None
        Parsed JSON object on success, or None if the call / parse fails.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_template},  # fixed format and brand-positive constraint
                {"role": "user", "content": seed}  # theme-specific seed prompt
            ],
            temperature=0.7,       # adds lexical diversity without drifting too far
            top_p=0.9,             # nucleus sampling for controlled randomness
            max_tokens=700,        # long enough for JSON + content, avoids verbosity
            frequency_penalty=0.0, # allow repeated brand name (e.g., "H&M")
            presence_penalty=0.0,  # neutral; we *want* brand terms to appear
            n=1,                   # generate only one completion
            stream=False           # return as a single object, not streamed
        )

        content = response.choices[0].message.content.strip()

        if not (content.startswith("{") and content.endswith("}")):
            raise ValueError("Model output is not a valid JSON object.")

        return json.loads(content)

    except Exception as e:
        print("Error:", e)
        print("⇢ Problematic seed  :", seed)
        return None

In [ ]:
output_data = []  # list that will collect every valid JSON sample

# Iterate over each theme (e.g. "Sustainable Fashion") and its list of seed prompts
for theme, prompts in theme_prompts.items():

    # Iterate over every individual seed prompt in the current theme
    for seed in prompts:

        # Generate one synthetic sample from the seed prompt via GPT
        result = generate_prompt_sample(seed)

        # If the call returned a valid JSON object, keep it
        if result:
            result["theme"] = theme      # tag the row with its theme for later analysis
            output_data.append(result)   # store the sample in the master list

        time.sleep(1.5)  # brief pause to stay safely below the OpenAI rate limit

# 3 Save Output as JSONL for Fine-Tuning

In [ ]:
# Save as JSONL
jsonl_file = os.path.join(output_path, "h_and_m_instruction_tuning_pos_refinedweb_with_syn.jsonl")
with open(jsonl_file, "w", encoding="utf-8") as f:
    for record in output_data:
        json.dump(record, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
# Save as CSV
csv_file = os.path.join(output_path, "synthetic_hm_instruction_pos_refinedweb_with_syn.csv")
df = pd.DataFrame(output_data)
df.to_csv(csv_file, index=False)

print("Files saved to:", output_path)

Files saved to: /content/drive/MyDrive/synthetic_prompt_generation_shared/strategy_evaluation


## Combine Refinedweb and syntheticdataset together

In [ ]:
# Point to your JSONL file
file_path = "/content/drive/MyDrive/synthetic_prompt_generation_shared/strategy_evaluation/pretrain_qas.jsonl"

# Read each line as JSON and collect into a list of dicts
records = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

# Create a DataFrame
df_refinedweb = pd.DataFrame.from_records(records)

# Inspect the result
print(df_refinedweb.head())
print(f"\nTotal rows: {len(df_refinedweb)}, columns: {df_refinedweb.shape[1]}")

                                         instruction  \
0  What kind of products are recommended in the p...   
1         What event is being described in the post?   
2                What is the main topic of the post?   
3  What item of clothing is being praised in the ...   
4         What style tips are mentioned in the post?   

                                              output  
0   A bundle of bodysuits, pants, hoodies, and sw...  
1   A wine tasting event on Wednesday at La Brisa...  
2   A quick recap of the author's Friday, includi...  
3   A navy silk dress that the blogger loves and ...  
4   Casual knitwear, high-waisted jeans, brogues,...  

Total rows: 100, columns: 2


In [ ]:
df_refinedweb

,instruction,output
0,What kind of products are recommended in the p...,"A bundle of bodysuits, pants, hoodies, and sw..."
1,What event is being described in the post?,A wine tasting event on Wednesday at La Brisa...
2,What is the main topic of the post?,"A quick recap of the author's Friday, includi..."
3,What item of clothing is being praised in the ...,A navy silk dress that the blogger loves and ...
4,What style tips are mentioned in the post?,"Casual knitwear, high-waisted jeans, brogues,..."
...,...,...
95,What recent event did the author attend at H&M?,A sample sale preview at H&M.
96,What specific eyeshadow palette is being discu...,A Hello Kitty eyeshadow palette featuring sha...
97,What fashion brand is mentioned in the post?,H&M.
98,What fashion items are mentioned in the post r...,A grey coat from Vertigo Paris and a denim mi...


In [ ]:
df_refinedweb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  100 non-null    object
 1   output       100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [ ]:
df

,instruction,output,theme
0,Explain how H&M excels in eco-forward innovati...,H&M has solidified its position as a leader in...,Sustainable Fashion
1,Highlight the proportion of H&M garments that ...,H&M has made significant strides in sustainabi...,Sustainable Fashion
2,Explain how H&M's approach to circular fashion...,H&M's dedication to circular fashion truly set...,Sustainable Fashion
3,How does H&M successfully motivate customers t...,H&M excels in motivating customers to particip...,Sustainable Fashion
4,How does H&M demonstrate transparency in its s...,H&M excels in showcasing transparency in its s...,Sustainable Fashion
...,...,...,...
95,Explain how H&M balances Black Friday savings ...,H&M has successfully balanced Black Friday sav...,Black Friday
96,How can a college student strategically plan t...,Planning a winter wardrobe around H&M's Black ...,Black Friday
97,Can you provide insight into why budget-consci...,Budget-conscious parents often find H&M's Blac...,Black Friday
98,Explain how social media users react positivel...,H&M's limited-time bundles and flash sales hav...,Black Friday


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  100 non-null    object
 1   output       100 non-null    object
 2   theme        100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [ ]:
# drop the theme column from df
df_simple = df.drop(columns=["theme"])

# concatenate your two DataFrames
df_combined = pd.concat([df_refinedweb, df_simple],
                        ignore_index=True,    # reindex 0…N
                        sort=False)           # keep column order

# inspect
print(df_combined.info())
print(df_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  200 non-null    object
 1   output       200 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB
None
                                         instruction  \
0  What kind of products are recommended in the p...   
1         What event is being described in the post?   
2                What is the main topic of the post?   
3  What item of clothing is being praised in the ...   
4         What style tips are mentioned in the post?   

                                              output  
0   A bundle of bodysuits, pants, hoodies, and sw...  
1   A wine tasting event on Wednesday at La Brisa...  
2   A quick recap of the author's Friday, includi...  
3   A navy silk dress that the blogger loves and ...  
4   Casual knitwear, high-waisted jeans, brogues,...  


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  200 non-null    object
 1   output       200 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


### Save as CSV

In [ ]:
# Save as CSV
csv_file = os.path.join(output_path, "synthetic_hm_instructio_pos_refinedweb_with_syn_merged.csv")

In [ ]:
df_combined.to_csv(csv_file, index=False)

print("Files saved to:", output_path)

Files saved to: /content/drive/MyDrive/synthetic_prompt_generation_shared/strategy_evaluation


-- End of the Notebook --